### Our main objective for this code is to find segments that vary in payment amounts/behaviour the assumption being that each has a stable segment level performance, which is a reasonable assumption because we have cohorts that are the same with respect to the segment/model score.

We are working with synthetic data, so there are distributions and such that doesn't follow the norm for real data, despite this - building the scaffolding for creating a CLV will still be pertinent.

Below is a class that loads the synthetic data, and maps the columns CustomerID,TransactionDate,TransactionAmount to the correct name. There are also methods in this class for creating RFM measures, RFM Segments, Demographics and Transaction Descriptor features (which I dont use), if you can do something useful with it then by all means.

Please Note that the dataloader, and rfm methods are general methods they can be used with any data that contains the columns CustomerID,TransactionID,TransactionDate,TransactionAmount.

I will close off by saying that the point of this exercise is to end off with segments that have stable performance at the "crowd" level. Thus nullifying the problem of high variance when trying to do a regression.

Once we have these segments, we have a choice of 2, we will then extrapolate payments per cohort/crowd/group. That will be the next step, this work will be carried out in a seperate notebook and class with written for the Vintage object with optimisation methods.

In [1]:
import pandas as pd
from pathlib import Path
import rfmbinner
import sklearn
import joblib
import xgboost as xgb
import numpy as np
import category_encoders as ce
import optuna
import random

class DataLoader:
    
    def __init__(self,path,customer_id,transaction_id,transaction_date,amount):
        """
        Initialize the DataLoader.

        Parameters:
            path (str): Path to the CSV file.
            customer_id (str): Column name for customer IDs.
            transaction_id (str): Column name for transaction IDs.
            transaction_date (str): Column name for transaction dates.
            amount (str): Column name for transaction amounts.
        
        """
        self.path = path
        self.customer_id = customer_id
        self.transaction_id = transaction_id
        self.transaction_date = transaction_date 
        self.amount = amount 
        
    def fetch_data(self) -> pd.DataFrame:
        """
        Fetch raw CSV data from the specified path.

        Parameters:
            path (str): Path to the CSV file.
        
        Returns:
            pd.DataFrame: Loaded data.
        """
        data_path = Path(self.path)
        if not data_path.exists():
            raise FileNotFoundError(f"CSV file not found at: {data_path}")
        data = pd.read_csv(data_path)
        data.rename(columns={self.customer_id:'CustomerID',
                             self.transaction_id: 'TransactionID',
                             self.transaction_date: 'Date',
                             self.amount: 'Amount'}, inplace=True)
        data['Date'] = pd.to_datetime(data['Date'],format='%Y-%m-%d')
        return data
    
    def calculate_rfm(self, snapshot_date: str, window: pd.Timedelta,df: pd.DataFrame) -> pd.DataFrame:
        """
        Calculate RFM metrics for customer segmentation.

        Parameters:
            snapshot_date (str): Date to calculate recency from (YYYY-MM-DD).
        
        Returns:
            pd.DataFrame: RFM metrics per CustomerID.
        """
        data = df.copy()
        snapshot = pd.to_datetime(snapshot_date)
        data['Date'] = pd.to_datetime(data.Date)
        data  = data[(data.Date<snapshot) & (data.Date>(snapshot-window))]
        data['recency'] = (snapshot - pd.to_datetime(data['Date'])).dt.days
        rfm = data.groupby('CustomerID').agg({
            'recency': 'min',
            'TransactionID': 'count',               # frequency: number of transactions
            'Amount': 'sum'             # monetary: total spend
        }).reset_index()

        rfm.rename(columns={'TransactionID': 'frequency', 'Amount': 'monetary'}, inplace=True)
        rfm['date'] = snapshot_date

        return rfm[['CustomerID', 'date', 'recency', 'frequency', 'monetary']]
    
    def calculate_target(self,date: str, window_size: pd.Timedelta, \
                         repurchase_threshold: float, df: pd.DataFrame) -> pd.DataFrame:
        """
        Calculating the target for the model
        Parameters:
            date (str): Date to calculate recency from (YYYY-MM-DD).
            window_size (pd.Timedelta): Number of days in which to consider repurchase
            repurchase_threshold: Minimum spend to be considered a repurchase
        Returns:
            pd.DataFrame: Target per CustomerID.
        """
        #caculate whether a customer made total purchases after date exceeding repurchase threshold
        data = df.copy()
        future_purchases = data[(data['Date'] > date) & \
                                (data['Date'] <= date + window_size)]

        target_customers = future_purchases.groupby('CustomerID')[['Amount']].sum()
        target_customers = target_customers[target_customers >= repurchase_threshold].index

        total_future_purchases = future_purchases.groupby('CustomerID')[['Amount']].sum()
        data = data.merge(total_future_purchases.rename(columns={'Amount': 'subsequent_purchases'}),
                            on='CustomerID', how='left')
        data['subsequent_purchases'] = data['subsequent_purchases'].fillna(0)

        data['target'] = data['CustomerID'].isin(target_customers).astype(int)
        data['date'] = date
        return data[['CustomerID','date','target','subsequent_purchases']].drop_duplicates()
    
    def rfm_segments(self,date: str, window: pd.Timedelta, df: pd.DataFrame) -> pd.DataFrame:
        """
        Segments customers based on their RFM score.

        Parameters:
            date (str): Date to calculate recency from (YYYY-MM-DD).
            df (pd.DataFrame): Raw transactional data.
        
        Returns:
            pd.DataFrame: RFM segments per CustomerID.
        """
        data = df.copy()
        data = self.calculate_rfm(date,window,data)
        # Create RFM scores
        data['r_score'] = pd.qcut(data['recency'].rank(method='first'), 5, labels=[5, 4, 3, 2, 1])
        data['f_score'] = pd.qcut(data['frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])
        data['m_score'] = pd.qcut(data['monetary'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])
        data['rfm_score'] = data['r_score'].astype(str) + \
                            data['f_score'].astype(str) + \
                            data['m_score'].astype(str)
        data['rfm_score_int'] = data['r_score'].astype(int) + \
                                data['f_score'].astype(int)+ \
                                data['m_score'].astype(int)           
        #create a dicitionary with the rfm_score that maps into segment, the key must be a regex
        segment_map = {
                        r'^55[1-5]$': 'Champions',
                        r'^54[1-5]$': 'Loyal Customers',
                        r'^45[1-5]$': 'Potential Loyalists',
                        r'^53[1-5]$': 'New or Returning Customers',
                        r'^33[1-5]$': 'Promising',
                        r'^22[1-5]$': 'Needs Attention',
                        r'^\d{3}$': 'Others'  # Matches any other 3-digit score
                    }
        data['segment'] = data['rfm_score'].replace(segment_map, regex=True)
        data['date'] = date
        return data[['CustomerID', 'date', 'rfm_score','rfm_score_int', 'segment']]
    
    def dedup_demographic_variables(self,df: pd.DataFrame) -> pd.DataFrame:
        """
        Deduplicate demographic variables for each customer, keeping the latest entry.
        
        Parameters:
            df (pd.DataFrame): Raw transactional data with demographic information.
        
        Returns:
            pd.DataFrame: Deduplicated demographic data per CustomerID.
        """
        data = df.copy()
        data['Date'] = pd.to_datetime(data['Date'],format='%m-%d-%Y %H:%M:%S')
        
        # Sort by CustomerID and Date to get the most recent demographic info
        data = data.sort_values(by=['CustomerID', 'Date'], ascending=True)
        
        # Drop duplicates, keeping the last (most recent) entry for each CustomerID
        # Assuming demographic variables are 'Gender', 'Age', 'Age Group', extend to the actual list
        demographic_cols = ['CustomerID', 'Gender', 'Age','Province']
        deduplicated_demographics = data[demographic_cols].drop_duplicates(subset=['CustomerID'], keep='last')
        
        return deduplicated_demographics
    
   


d:\VS CODE PROJECTS\CLVM\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


For the code below the process is as follows:

1. Initialise a dataloader
2. Fetch the synthetic data
3. Run the appropriate methods of the dataloader in order to create a collection of features
4. We join all the features together and create a "model_data" dataframe.

In [2]:
params = {'path':r'data\customer_transaction_data.csv',
            'customer_id':'CustomerID',
            'transaction_id':'TransactionID',
            'transaction_date':'PurchaseDate',
            'amount':'TotalAmount'}
loader = DataLoader(**params)
data = loader.fetch_data()

In [3]:
data['Date'].agg(['min','max'])

min   2020-08-09
max   2025-08-09
Name: Date, dtype: datetime64[ns]

In [4]:
start_date = '2022-08-30'
end_date = pd.to_datetime('2025-09-26')
pivot_date = pd.to_datetime(start_date)
rfms = []
targets = []
rfm_segments = []
transaction_descriptor_data = []

while pivot_date<end_date:
    rfms.append(loader.calculate_rfm(pivot_date,pd.Timedelta(days=99999),data))
    rfm_segments.append(loader.rfm_segments(pivot_date,pd.Timedelta(days=99999),data))
    targets.append(loader.calculate_target(pivot_date,pd.Timedelta(weeks=56),0,data))
    pivot_date+=pd.Timedelta(weeks=2)

rfm_data = pd.concat(rfms)
target_data = pd.concat(targets)
rfm_segments = pd.concat(rfm_segments)
dem_data = loader.dedup_demographic_variables(data)

In [5]:
len(data)

100000

In [6]:
model_data = rfm_data.merge(target_data,how='left',left_on=['CustomerID','date'],right_on=['CustomerID','date'])
model_data = model_data.merge(dem_data,how='left',left_on='CustomerID',right_on='CustomerID')
#model_data = model_data.merge(transaction_descriptor_data,how='left', left_on=['CustomerID','date'],right_on=['CustomerID','date'])
model_data = model_data.merge(rfm_segments.drop('segment',axis=1),how='left', left_on=['CustomerID','date'],right_on=['CustomerID','date'])

In [7]:
end_date_of_training = '2024-07-31'
model_data = model_data.set_index(['CustomerID','date']).drop('rfm_score',axis=1)
model_data = model_data[model_data.index.get_level_values(1)<end_date_of_training]
model_data.columns

Index(['recency', 'frequency', 'monetary', 'target', 'subsequent_purchases',
       'Gender', 'Age', 'Province', 'rfm_score_int'],
      dtype='object')

In [8]:
model_data.target.value_counts()/len(model_data)

target
0    0.882471
1    0.117529
Name: count, dtype: float64

In [9]:
random.seed(1234)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    model_data.drop(['target', 'subsequent_purchases'], axis=1),
    model_data['target'],
    test_size=0.2,
    random_state=42
)

for col in X_train.columns:
    sample = X_train[col].iloc[0]
    if isinstance(sample, (list, np.ndarray)):
        print(f"Column '{col}' contains unhashable types: {type(sample)}")

encoder = ce.TargetEncoder()
X_train = encoder.fit_transform(X_train, y_train)
X_test = encoder.transform(X_test)
joblib.dump(encoder,r'data\encoder.pkl')

# Optuna + XGBoost training
def train_xgboost_with_optuna(X, y, n_trials=10, test_size=0.2, random_state=42):
    X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    def objective(trial):
        params = {
            "verbosity": 0,
            "objective": "binary:logistic",
            "eval_metric": "logloss",
            "booster": trial.suggest_categorical("booster", ["gbtree", "dart","gblinear"]),
            "lambda": trial.suggest_float("lambda", 1e-8, 10.0, log=True),
            "alpha": trial.suggest_float("alpha", 1e-8, 10.0, log=True),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "eta": trial.suggest_float("eta", 0.01, 0.3),
            "scale_pos_weight": trial.suggest_float("scale_pos_weight",0,1)
        }

        dtrain = xgb.DMatrix(X_train, label=y_train)
        dval = xgb.DMatrix(X_val, label=y_val)
        model = xgb.train(params, dtrain, num_boost_round=50,
                          evals=[(dval, "validation")],
                          early_stopping_rounds=10,
                          verbose_eval=False)
        preds = model.predict(dval)
        return sklearn.metrics.roc_auc_score(y_val, preds)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    best_params = study.best_params
    best_params.update({
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "logloss"
    })

    final_model = xgb.train(best_params, xgb.DMatrix(X, label=y), num_boost_round=study.best_trial.number)
    return final_model, study

# Train and evaluate
model, study = train_xgboost_with_optuna(X_train, y_train)
y_probs = model.predict(xgb.DMatrix(X_test))

print(sklearn.metrics.classification_report(y_test, y_probs > 0.5))
print("ROC AUC:", sklearn.metrics.roc_auc_score(y_test, y_probs))


[I 2025-08-15 14:27:48,612] A new study created in memory with name: no-name-359f6562-1dc1-4226-b26b-d33fddd10d56
[I 2025-08-15 14:27:50,750] Trial 0 finished with value: 0.6914685311891859 and parameters: {'booster': 'gbtree', 'lambda': 3.044521232030346e-08, 'alpha': 0.0003580644715811766, 'subsample': 0.5303498581700106, 'colsample_bytree': 0.6623721163440668, 'max_depth': 7, 'eta': 0.056372917722366606, 'scale_pos_weight': 0.5690651187747088}. Best is trial 0 with value: 0.6914685311891859.
[I 2025-08-15 14:27:51,407] Trial 1 finished with value: 0.6288116747102028 and parameters: {'booster': 'gblinear', 'lambda': 0.0007284735691032068, 'alpha': 2.4815841293405826e-06, 'subsample': 0.6052854961699895, 'colsample_bytree': 0.5957189064986969, 'max_depth': 3, 'eta': 0.2532013050817113, 'scale_pos_weight': 0.18942037458824124}. Best is trial 0 with value: 0.6914685311891859.
[I 2025-08-15 14:27:53,246] Trial 2 finished with value: 0.6637371798632562 and parameters: {'booster': 'dart', 

              precision    recall  f1-score   support

           0       0.89      1.00      0.94    428015
           1       0.83      0.04      0.08     56884

    accuracy                           0.89    484899
   macro avg       0.86      0.52      0.51    484899
weighted avg       0.88      0.89      0.84    484899

ROC AUC: 0.6856625159942485


In [10]:
sklearn.metrics.roc_auc_score(y_test,model.predict(xgb.DMatrix(X_test)))

0.6856625159942485

In [11]:
X_test['QSegment'],score_bins= pd.cut(y_probs,bins=[-np.inf,0.11,0.12,np.inf],retbins=True)
joblib.dump(score_bins,r'data\score_bins.pkl')

['data\\score_bins.pkl']

In [12]:
summary_data = X_test.merge(target_data,how='left', left_on=['CustomerID','date'], right_on=['CustomerID','date'])
summary = summary_data.groupby('QSegment').agg({'recency':'count','monetary':'sum','target':'sum','subsequent_purchases':'sum'})
summary = summary.rename(columns = {'recency':'Count','monetary':'prior_purchases','target':'Purchase Rate'})
summary['prior_purchases in ZAR'] = summary.prior_purchases.apply(lambda x: f'R {x:,.2f}')
summary['Value %'] = (summary['prior_purchases']/summary['prior_purchases'].sum()*100).apply(lambda x: f'{x:.2f}%')
summary['AVG subs. spend in ZAR'] = (summary['subsequent_purchases']/summary.Count).apply(lambda x: f'R {x:.2f}')
summary['subsequent_purchases in ZAR (3 month window)'] = summary.subsequent_purchases.apply(lambda x: f"R {x:,.2f}")
summary['Purchase Rate'] =  (summary['Purchase Rate']/summary.Count).apply( lambda x: f'{x:.2%}')
summary.drop(['prior_purchases','subsequent_purchases'],axis=1)

C:\Users\wesle\AppData\Local\Temp\ipykernel_8920\238198811.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = summary_data.groupby('QSegment').agg({'recency':'count','monetary':'sum','target':'sum','subsequent_purchases':'sum'})


,Count,Purchase Rate,prior_purchases in ZAR,Value %,AVG subs. spend in ZAR,subsequent_purchases in ZAR (3 month window)
QSegment,,,,,,
"(-inf, 0.11]",441167,9.06%,"R 58,449,025.32",66.59%,R 15.13,"R 6,674,669.68"
"(0.11, 0.12]",8058,22.82%,"R 2,759,923.35",3.14%,R 53.42,"R 430,449.71"
"(0.12, inf]",35674,42.29%,"R 26,568,251.24",30.27%,R 138.25,"R 4,932,068.02"


In [13]:
summary_data = X_test.merge(target_data,how='left', left_on=['CustomerID','date'], right_on=['CustomerID','date'])
summary = summary_data.groupby(['rfm_score_int']).agg({'recency':'count','monetary':'sum','target':'sum','subsequent_purchases':'sum'})
summary = summary.rename(columns = {'recency':'Count','monetary':'prior_purchases','target':'Purchase Rate'})
summary['prior_purchases in ZAR'] = summary.prior_purchases.apply(lambda x: f'R {x:,.2f}')
summary['Value %'] = (summary['prior_purchases']/summary['prior_purchases'].sum()*100).apply(lambda x: f'{x:.2f}%')
summary['AVG subs. spend in ZAR'] = (summary['subsequent_purchases']/summary.Count).apply(lambda x: f'R {x:.2f}')
summary['subsequent_purchases in ZAR (3 month window)'] = summary.subsequent_purchases.apply(lambda x: f"R {x:,.2f}")
summary['Purchase Rate'] =  (summary['Purchase Rate']/summary.Count).apply( lambda x: f'{x:.2%}')
summary.drop(['prior_purchases','subsequent_purchases'],axis=1)

,Count,Purchase Rate,prior_purchases in ZAR,Value %,AVG subs. spend in ZAR,subsequent_purchases in ZAR (3 month window)
rfm_score_int,,,,,,
3,5404,8.14%,"R 109,079.32",0.12%,R 12.52,"R 67,672.48"
4,15422,8.02%,"R 521,860.06",0.59%,R 13.40,"R 206,710.74"
5,29059,8.27%,"R 1,465,486.20",1.67%,R 13.03,"R 378,765.21"
6,47067,7.73%,"R 3,310,910.84",3.77%,R 12.24,"R 576,140.59"
7,61423,7.50%,"R 5,696,871.95",6.49%,R 11.48,"R 705,198.68"
8,66861,8.14%,"R 8,023,284.60",9.14%,R 12.75,"R 852,480.61"
9,63893,8.30%,"R 9,225,249.60",10.51%,R 13.39,"R 855,376.33"
10,55189,9.37%,"R 9,437,714.52",10.75%,R 16.15,"R 891,531.20"
11,43152,11.54%,"R 8,882,400.54",10.12%,R 19.36,"R 835,498.79"


In [14]:
pd.DataFrame(data = zip(X_train.columns,model.get_score(importance_type='gain').values()),columns=['Features','Importances'])

,Features,Importances
0,recency,18.056435
1,frequency,2354.354004
2,monetary,298.917480
3,Gender,11.020784
4,Age,14.183438
5,Province,13.424657
6,rfm_score_int,676.188965


In [15]:
model.save_model(r'data\PropensityToBuy.json')